## Retrieve red links considering oldest reference pages (among main pages and redirections)

In [1]:
import mwparserfromhell
import numpy as np
import pandas as pd
import re
from mw.xml_dump import Iterator
from mw import Timestamp

In [2]:
from collections import defaultdict

In [3]:
# to make interactive plotting possible
%matplotlib inline

# for auto-reloading external modules
%load_ext autoreload
%autoreload 2

# make plots a bit nicer
import matplotlib.pyplot as plt
plt.matplotlib.rcParams.update({'font.size': 18, 'font.family': 'serif'})

In [4]:
from datetime import date, time, datetime

In [5]:
import matplotlib.patches as mpatches

## helpers

In [6]:
def multiple_replace(text):
    
    adict = {'[[': '', ']]': '', ' ': '_'}
    
  # Create a regular expression from all of the dictionary keys
    regex = re.compile("|".join(map(re.escape, adict.keys(  ))))

  # For each match, look up the corresponding value in the dictionary
    sub_text = regex.sub(lambda match: adict[match.group(0)], text)

    first_title = re.split("\|", sub_text)
    
    return first_title[0]

In [7]:
def aux_fun(x):
    try:
        return Timestamp(np.int64(x))
    except:
        print(x)
        return None

In [8]:
def link_status(rvtime, page_creation_time):
    if pd.isnull(page_creation_time) or page_creation_time > rvtime:
        return 'red link'
    if page_creation_time <= rvtime:
        return 'blue link'

### create a table of all pages and redirection with min time of page creation

In [ ]:
!RUN ONCE

# load a table with page creation time for ukwiki
page_creation_df = pd.read_csv('../data/'+'timestamp-for-first-revision-of-all-articles-in-ukwiki.tsv', sep = '\t',)

# load a table with redirects in ukwiki
redirects_df = pd.read_csv('../data/'+'ukredirect.tsv', sep = '\t',)

redirects_df.columns = ['rd_id', 'ref_title', 'rd_interwiki']

redirects_df = redirects_df[['rd_id', 'ref_title']]

pagestamp_with_redirects_df = page_creation_df.merge(redirects_df, left_on='page_id', right_on='rd_id', how='outer')

pagestamp_with_redirects_df.columns = ['page_creation_timestamp', 'ref_title', 'page_id', 'rd_id', 'main_page_title']

pagestamp_with_redirects_df = pagestamp_with_redirects_df[pagestamp_with_redirects_df['page_creation_timestamp'].notnull()]

pagetime_redirects_df = pagestamp_with_redirects_df[pagestamp_with_redirects_df['rd_id'].notnull()]

pagetime_main_df = pagestamp_with_redirects_df[pagestamp_with_redirects_df['rd_id'].isnull()]

pagetime_main_df['main_page_title'] = pagetime_main_df[['main_page_title']].apply(lambda x: pagetime_main_df['ref_title'], axis=0)

final_pagetime_df = pd.concat([pagetime_redirects_df, pagetime_main_df])

final_pagetime_df.page_creation_timestamp = final_pagetime_df.page_creation_timestamp.apply(lambda x: Timestamp(np.int64(x)))

min_pagetime_df = final_pagetime_df.groupby(['main_page_title'])['page_creation_timestamp'].min()

min_pagetime_df = min_pagetime_df.to_frame()

min_pagetime_df.reset_index(level=0, inplace=True)

final_pagetime_df = final_pagetime_df.merge(min_pagetime_df, on='main_page_title', how='left')

del final_pagetime_df['page_creation_timestamp_x']

final_pagetime_df.columns = ['ref_title', 'page_id', 'rd_id', 'main_page_title', 'min_pagecreation_timestamp']

final_pagetime_df.to_csv('../working_files/'+'final_pagetime_df.tsv', index=False, sep = '\t')

## get red links

In [13]:
page_creation_df = pd.read_csv('../working_files/'+'final_pagetime_df.tsv', sep = '\t')

In [14]:
page_creation_df.head()

,ref_title,page_id,rd_id,main_page_title,min_pagecreation_timestamp
0,!,829961.0,829961.0,Знак_оклику,2.008033e+13
1,!!,2062457.0,2062457.0,Факторіал,2.004041e+13
2,!!!Fuck_You!!!_and_Then_Some,2591631.0,2591631.0,Fuck_You_and_Then_Some,2.015062e+13
3,!Action_Pact,1767358.0,1767358.0,!Action_Pact!,2.010111e+13
4,!T.O.O.H.,1767359.0,1767359.0,!T.O.O.H.!,2.009033e+13


### Check for chosen pages

In [ ]:
!RUN ONCE

noise_pages = ('Головна сторінка', 'Обговорення:', 'Обговорення користувача:', 'Обговорення Вікіпедії:', 'Категорія:', 'Шаблон:', 'Вікіпедія:',
               'Файл:','Користувач:', 'Категорія:Списки:', 'Портал:', 'Довідка:', ':Категорія')

# Construct dump file iterator
dump = Iterator.from_file(open("/media/andrii/earth/Katia/CS/!redlinks_project/red-links-v2.0/data/ukwiki_dumps/ukwiki-20200501-pages-meta-history1.xml-p1p17370"))

# Iterate through i pages
stopwords = ['[[Категорія:', '[[Файл:']
revs_links_time = []

i = 1000
for page in dump:
    if i == 0:
        break
    elif page.title.startswith(noise_pages):
        continue
    elif page.redirect is not None:
        continue
    else:
        print('PAGE TITLE:', page.title)
        for revision in page:
            timestamp = revision.timestamp
            revision = revision.text
            get_revision = mwparserfromhell.parse(revision)
            links = get_revision.filter_wikilinks()
            filtered = []
            for l in links:
                if not any([sw in l for sw in stopwords]):
                    revs_links_time.append((str(page.title), str(l), timestamp))
    i -= 1

In [32]:
# get latest revisions of pages

noise_pages = ('Головна сторінка', 'Обговорення:', 'Обговорення користувача:', 'Обговорення Вікіпедії:', 'Категорія:', 'Шаблон:', 'Вікіпедія:',
               'Файл:','Користувач:', 'Категорія:Списки:', 'Портал:', 'Довідка:', ':Категорія')

# Construct dump file iterator
dump = Iterator.from_file(open("/media/andrii/earth/Katia/CS/!redlinks_project/red-links-v2.0/data/ukwiki_dumps/ukwiki-20200501-pages-meta-history1.xml-p1p17370"))

# Iterate through i pages
stopwords = ['[[Категорія:', '[[Файл:']
revs_time = {}

i = 1000
for page in dump:
    page_revisions = []
    if i == 0:
        break
    elif page.title.startswith(noise_pages):
        continue
    elif page.redirect is not None:
        continue
    else:
        page_title = page.title
        print('PAGE TITLE:', page_title)
        for revision in page:
            timestamp = revision.timestamp
            page_revisions.append(timestamp)
    revs_time[page_title] = page_revisions
    i -= 1

latest_revisions = {}
for k, v in revs_time.items():
    latest_revisions[k] = max(v)

latest_revisions_df = pd.DataFrame.from_dict(latest_revisions, orient='index')

latest_revisions_df.columns = ['rvtimestamp']

latest_revisions_df.to_csv(('../working_files/'+'latest_revisions_df.csv'))

PAGE TITLE: Географія
PAGE TITLE: Атом
PAGE TITLE: Мільярд
PAGE TITLE: Ядро
PAGE TITLE: Мільйон
PAGE TITLE: Хімія
PAGE TITLE: Діаметр
PAGE TITLE: Метр
PAGE TITLE: Біологія
PAGE TITLE: Коло
PAGE TITLE: Довжина
PAGE TITLE: Точка
PAGE TITLE: Крапка
PAGE TITLE: Речення
PAGE TITLE: Слово
PAGE TITLE: Мова
PAGE TITLE: Радіус
PAGE TITLE: Есперанто
PAGE TITLE: 1944
PAGE TITLE: 1854
PAGE TITLE: 1912
PAGE TITLE: 1929
PAGE TITLE: 1999
PAGE TITLE: Високович Володимир Костянтинович
PAGE TITLE: Молостова Ірина Олександрівна
PAGE TITLE: 1834
PAGE TITLE: 1908
PAGE TITLE: Антонович Володимир Боніфатійович
PAGE TITLE: 1904
PAGE TITLE: 1985
PAGE TITLE: Комар Антон Пантелеймонович
PAGE TITLE: Фізик
PAGE TITLE: Фізика
PAGE TITLE: Закон (право)
PAGE TITLE: 1961
PAGE TITLE: Довгаль Олександр Михайлович
PAGE TITLE: 1839
PAGE TITLE: 1884
PAGE TITLE: Чубинський Павло Платонович
PAGE TITLE: Гімн України
PAGE TITLE: Рудольф Людвіг Мессбауер
PAGE TITLE: 1979
PAGE TITLE: Омеляновський Михайло Еразмович
PAGE TITLE: У

PAGE TITLE: Korn-shell
PAGE TITLE: Римська система числення
PAGE TITLE: Фізичні константи
PAGE TITLE: Періодична система хімічних елементів
PAGE TITLE: Освіта
PAGE TITLE: Український вільний університет
PAGE TITLE: 1799
PAGE TITLE: 1600
PAGE TITLE: 1601
PAGE TITLE: 1602
PAGE TITLE: 1603
PAGE TITLE: 1604
PAGE TITLE: 1605
PAGE TITLE: 1606
PAGE TITLE: 1607
PAGE TITLE: 1608
PAGE TITLE: 1609
PAGE TITLE: 1610
PAGE TITLE: 1611
PAGE TITLE: 1612
PAGE TITLE: 1613
PAGE TITLE: 1614
PAGE TITLE: 1615
PAGE TITLE: 1616
PAGE TITLE: 1617
PAGE TITLE: 1618
PAGE TITLE: 1619
PAGE TITLE: 1620
PAGE TITLE: 1621
PAGE TITLE: 1622
PAGE TITLE: 1623
PAGE TITLE: 1624
PAGE TITLE: 1625
PAGE TITLE: 1626
PAGE TITLE: 1627
PAGE TITLE: 1628
PAGE TITLE: 1629
PAGE TITLE: 1630
PAGE TITLE: 1631
PAGE TITLE: 1632
PAGE TITLE: 1633
PAGE TITLE: 1634
PAGE TITLE: 1635
PAGE TITLE: 1636
PAGE TITLE: 1637
PAGE TITLE: 1638
PAGE TITLE: 1639
PAGE TITLE: 1640
PAGE TITLE: 1641
PAGE TITLE: 1642
PAGE TITLE: 1643
PAGE TITLE: 1644
PAGE TITLE: 164

PAGE TITLE: Історія України
PAGE TITLE: Бандера Степан Андрійович
PAGE TITLE: 31 січня
PAGE TITLE: 1 лютого
PAGE TITLE: 2 лютого
PAGE TITLE: 3 лютого
PAGE TITLE: 4 лютого
PAGE TITLE: 5 лютого
PAGE TITLE: 6 лютого
PAGE TITLE: 7 лютого
PAGE TITLE: 8 лютого
PAGE TITLE: 9 лютого
PAGE TITLE: 10 лютого
PAGE TITLE: 11 лютого
PAGE TITLE: 12 лютого
PAGE TITLE: 13 лютого
PAGE TITLE: 14 лютого
PAGE TITLE: 15 лютого
PAGE TITLE: 16 лютого
PAGE TITLE: 17 лютого
PAGE TITLE: 18 лютого
PAGE TITLE: 19 лютого
PAGE TITLE: 20 лютого
PAGE TITLE: 21 лютого
PAGE TITLE: 22 лютого
PAGE TITLE: 23 лютого
PAGE TITLE: 24 лютого
PAGE TITLE: 25 лютого
PAGE TITLE: 26 лютого
PAGE TITLE: 27 лютого
PAGE TITLE: 28 лютого
PAGE TITLE: 29 лютого
PAGE TITLE: 1 березня
PAGE TITLE: 2 березня
PAGE TITLE: 3 березня
PAGE TITLE: 4 березня
PAGE TITLE: 5 березня
PAGE TITLE: 7 березня
PAGE TITLE: 11 березня
PAGE TITLE: 12 березня
PAGE TITLE: 13 березня
PAGE TITLE: 14 березня
PAGE TITLE: 15 березня
PAGE TITLE: 16 березня
PAGE TITLE: 17

In [13]:
# create dataframe of all links from all revisions of the chosen pages
revision_links_df = pd.DataFrame(revs_links_time, columns=['parent_page', 'link', 'rvtimestamp'])
revision_links_df['link_title'] = revision_links_df['link'].apply(lambda row: multiple_replace(row))
revision_links_df['link_title'] = revision_links_df['link_title'].str.lower()

In [14]:
revision_links_df

,parent_page,link,rvtimestamp,link_title
0,Географія,[[uk:Географя]],20040126110432,uk:географя
1,Географія,[[af:Geografie]],20040126110432,af:geografie
2,Географія,[[als:Geographie]],20040126110432,als:geographie
3,Географія,[[ar:إستونيا]],20040126110432,ar:إستونيا
4,Географія,[[bg:География]],20040126110432,bg:география
...,...,...,...,...
194562,Атом,[[Референтний атом]],20200429131823,референтний_атом
194563,Атом,[[Лігатний атом]],20200429131823,лігатний_атом
194564,Атом,[[Спіроатом]],20200429131823,спіроатом
194565,Атом,[[Атом Бора]],20200429131823,атом_бора


In [ ]:
# get the latest revisions of pages

latest_revisions = revision_links_df.groupby(['parent_page'], sort=False)['rvtimestamp'].max()

In [ ]:
latest_revisions

In [ ]:
latest_revisions_df = latest_revisions.to_frame()
latest_revisions_df.to_csv('../working_files/'+'latest_revisions_df.csv', sep='^')

In [ ]:
!Run Once
# creates a file of links from all revisions of the chosen pages
revision_links_df.to_pickle('/media/andrii/earth/Katia/CS/!redlinks_project/red-links-v2.0/working_files/'+'revision_links_1000pages.pkl')

In [15]:
revision_links_df = pd.read_pickle('/media/andrii/earth/Katia/CS/!redlinks_project/red-links-v2.0/working_files/'+'revision_links_1000pages.pkl')

In [ ]:
# get links from the revisions of a certain time period
revision_links_df_after2010 = revision_links_df.drop(revision_links_df[revision_links_df.rvtimestamp.map(lambda x: int(x.strftime("%Y")) < 2010)].index)

In [ ]:
# create a table of links with their status (red/blue/check)

page_creation_df['ref_title'] = page_creation_df['ref_title'].str.lower()
page_creation_df['main_page_title'] = page_creation_df['main_page_title'].str.lower()

link_df = revision_links_df_after2010.merge(page_creation_df, left_on='link_title', right_on='ref_title', how='left')

del link_df['ref_title']

# clean noise links
noise_langlinks = ('af:', 'als:', 'ar', 'bg:', 'bs:', 'ca:', 'cs:', 'da:', 'de:', 'el:', 'en:', 'es:', 'et:',
                   'eu:', 'fi:', 'fr:', 'ga:', 'hi:', 'hr:', 'hu:', 'ia:', 'is:', 'it:', 'ja:', 'ko:', 'la:',
                   'lt:', 'nl:', 'no:', 'oc:', 'pl:', 'pt:', 'ro:', 'ru:', 'sk:', 'sl:', 'sq:', 'sr:', 'sv:',
                   'sw:', 'tr:', 'uk:', 'zh:', 'co:', 'cy:', 'eo:', 'fy:', 'gl:', 'he:', 'id:', 'ks:', 'lv:',
                   'ms:', 'nds:', 'simple:', 'th:', 'category:', 'ast:', 'az:', 'csb:', 'ie:', 'lb:', 'te:',
                   'tl:', 'ur:', 'vi:', 'vo:', 'zh-cn:', 'image:', 'be:', 'bn:', 'br:', 'io:', 'ka:', 'mg:',
                   'mk:', 'mt:', 'rm:', 'scn:', 'ab:', 'fo:', 'fur:', 'li:', 'ab:', 'an:', 'ba:', 'ky:', 'nn:',
                   'os:', 'kv:', 'sc:', 'ceb:', 'lad:', 'mh:', 'gu:', 'ta:', 'to:', 'tk:', 'зображення:', 'av:',
                   'bat-smg:', 'bm:', 'cv:', 'ku:', 'kw:', 'lmo:', 'nrm:', 'ps:', 'rmy:', 'sh:', 'tg:', 'ug:',
                   'uz:', 'vec:', 'zh-yue:', 'ilo:', 'bar:', 'iu:', 'nov:', 'wa:', 'qu:', 'wa:', 'my:', 'frp:',
                   'hsb:', 'nah:', 'yi:', 'ht:', 'mr:', 'ng:', 'sco:', 'su:', 'be-x-old:', 'ne:', 'km:', 'mn:',
                   'sm:', 'fa:', 'lij:', 'map-bms:', 'ml:', 'ru-sib:', 'jv:', 'wo:', 'tpi:', 'as:', 'tpi:', 'lo:',
                   'bh:', 'war:', 'mzn:', 'pms:', 'stq:', 'szl:', 'ay:', 'si:', 'fiu-vro:', 'ln:', 'bcl:', 'ln:',
                   'dv:', 'gn:', 'pam:', 'so:', 'zea:', 'ext:', 'gan:', 'hy:', 'sah:', 'ak:', 'kab:', 'nds-nl:',
                   'mdf:', 'ts:', 'gv:', 'pnt:', 'kk:', 'zh-min-nan:', 'jbo:', 'kn:', 'kl:', 'mwl:', 'hif:', 'mhr:',
                   'yo:', 'new:', 'pcd:', 'tt:', 'krc:', 'am:', 'frr:', 'ig:', 'pnb:', 'bo:', 'bi:', 'ss:', 'rue:',
                   'kbd:', 'zh-classical:', 'sn:', 'sa:', 'roa-tara:', 'gd:', 'cbk-zam:', 'lez:', 'nso:', 'ckb:',
                   'vep:', 'pa:', 'file:', 'kaa:', 'pfl:', 'haw:', 'pih:', 'mo:', 'ee:', 'xh:', 'srn:', 'wuu:',
                   'ksh:', 'ki:', 'szl:', 'crh:', 'zh-tw:', 'dsb:', 'lg:', 'koi:', 'xal:', 'jp:', 'pih:', 'hak:',
                   ':категорія:', 'cu:', ':ru:', 'ff:', 'wikipedia:en:', 'bug:', 'xmf:', 'ce:', 'diq:', 'vls:',
                   'bsd', 'myv:', 'bjn:', 'or:', 'ti:', 'cdo:', 'mi:', 'pap:', 'se:', 'історія_науки#',
                   'planetmath:', 'pi:', 'ty:', 'tet:', 'bpy:', 'nap:', 'gag:', 'ang:', 'udm:',
                   'zu:', 'rw:', 'lbe:', 'za:', 'ha:', 'ch:', 'nb:', 'min:', 'bxr:', 'ace:', 'pag:',
                   'вікіпедія:', 'спеціальна:', 'користувач:', 'ik:', 'roa-rup:', 'ltg:', 'tlh:', 'got:', 'rn:',
                   'tokipona:', 'minnan:', 'довідка:', 'nv:', 'eml:', 'na:', 'kg:', 'sd:', 'wikipedia:', 'chr:',
                   'pdc:', 'fj:', 'портал:')

link_df_clean = link_df.drop(link_df[link_df.link_title.astype(str).str.startswith(noise_langlinks)].index)

link_status_df = link_df_clean.drop(link_df_clean[link_df_clean.link_title.map(lambda x: '#' in str(x))].index)

link_status_df.min_pagecreation_timestamp =\
link_status_df.min_pagecreation_timestamp.apply(lambda x: Timestamp(np.int64(x)) if (pd.notnull(x)) else x)

link_status_df['link_status'] = link_status_df.apply(lambda x: link_status(x['rvtimestamp'], x['min_pagecreation_timestamp']), axis=1)

In [ ]:
# write a table with link status to file

link_status_df.to_pickle('/media/andrii/earth/Katia/!redlinks_project/'+'link_status_1000pages_after2010.pkl')
#unpickled_df = pd.read_pickle("./dummy.pkl")

In [8]:
# get a table only with red links and write it to file
df_red = link_status_df.loc[link_status_df['link_status'] == 'red link']

df_red['rvtimestamp'] = df_red.rvtimestamp.apply(lambda x: datetime.fromtimestamp(x).date().replace(day=1))

df_red_less = df_red[['parent_page', 'rvtimestamp', 'link_title', 'link_status']]


df_red_less_unique = df_red_less.drop_duplicates()

df_red_less_unique.to_csv('../'+'df_red_less_unique.csv', index=False, sep='^')

/home/andrii/virtenvs/tb/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [49]:
# count number of unique red links for a revision of a page

df_red_less_unique_time = df_red_less_unique_time.drop(df_red_less_unique_time[df_red_less_unique_time.rvtimestamp.map(lambda x: int(x.split('-')[0]) < 2010)].index)

df_red_less_unique_time = df_red_less_unique.groupby(['parent_page', 'rvtimestamp']).size().reset_index(name='counts')

df_red_less_unique_time.to_csv('../'+'df_red_less_unique_time.csv', index=False, sep='^')